# Analyze different architectures for **License plate detection** using **Yolo**

We define the following pipeline:
- A yolo model pretrained on *COCO 2017* dataset and fine-tuned using an additional dataset for detection of cars, trucks, pedestrians and traffic lights (this model will be used to extract region of interest (ROI) from a frame);
- A yolo model pretrained on *COCO 2017* dataset and fine-tuned using a new dataset for license plate detection;
- **maybe** a model using *particle filters* used to track identified license plate from different frames without apply tracking on consecutive frames;
- A convolutional neural network used for character recognition from license plate.

### 1. Vehicle detection model

First of all, we must install **ultralytics** package and load a nano version of **Yolov8** model, pretrained on *COCO 2017* dataset.

In [1]:
%pip install ultralytics
from ultralytics import YOLO
import torch

print(torch.cuda.is_available())

model = YOLO('yolov8n.pt')
model.cuda()

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/58/df/b44a846a103476fe9b4f9bb0fe99cb787fecb48ee795c2b2c4578ca98217/ultralytics-8.1.27-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 13.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
False


AssertionError: Torch not compiled with CUDA enabled

Then we train the model using this configuration:
* *epochs* = 100 (number of epochs for training)
* *imgsz* = 640 (set height of image to 640px)
* *batch* = -1 (allow package to choose optimal batch size)
* *pretrained* = True (fine-tune model using pretrained weights)

In [3]:
# Train the model
results = model.train(data='/kaggle/input/selfdrive-object-detection/data.yaml', epochs=100, imgsz=640, batch=-1, plots=True, pretrained=True)

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/selfdriving-object-detection/data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=Fal

100%|██████████| 755k/755k [00:00<00:00, 38.5MB/s]
2024-03-11 13:28:45,988	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-11 13:28:46,481	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...


100 epochs completed in 4.532 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.26 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:18<00:00,  1.23s/it]


                   all       1980      14112      0.808      0.634      0.713        0.4
                 biker       1980        258      0.669      0.539      0.603      0.313
                   car       1980       9269      0.837      0.761      0.811      0.526
            pedestrian       1980       1481      0.692      0.499      0.558      0.263
          trafficLight       1980        398      0.817      0.797      0.827      0.508
    trafficLight-Green       1980        756      0.801      0.595      0.678      0.298
trafficLight-GreenLeft       1980         47      0.892      0.468      0.679      0.342
      trafficLight-Red       1980       1049      0.844      0.744      0.795      0.463
  trafficLight-RedLeft       1980        265      0.883      0.686      0.801      0.454
   trafficLight-Yellow       1980         35      0.814      0.486      0.568      0.277
                 truck       1980        554      0.828      0.765      0.811       0.56
Speed: 0.4ms preproce

wandb: Network error (ConnectionError), entering retry loop.


lr/pg0,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▂▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇████████████████████
metrics/mAP50-95(B),▁▂▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████
metrics/precision(B),▁▁▆▅▆▅▆▆▅▅▆▅▇▆▆▆▇▇▇█▇▇▇▇█▇█▇▇█▇▇██████▇█
metrics/recall(B),▁▁▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███▇▇███████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁


**Maybe** we give up this method and use a trained model from Roboflow :). Let's load this model and view results on test data (No, not a good idea :) ).

So, we are back and we will upload the model weights to roboflow dataset.

In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="IUxYJzulWVkv30nEW5J0")
project = rf.workspace("selfdriving-traffic-detection").project("self-driving-traffic-detection")
version = project.version(2)

version.deploy("yolov8", "/kaggle/working/runs/detect/train")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.26, to fix: `pip install ultralytics==8.0.196`


Would you like to continue with the wrong version of ultralytics? y/n:  y


View the status of your deployment at: https://app.roboflow.com/selfdriving-traffic-detection/self-driving-traffic-detection/2
Share your model with the world at: https://universe.roboflow.com/selfdriving-traffic-detection/self-driving-traffic-detection/model/2


Next, we try to test model using videos with different scenarios. Here are some examples: different car speeds (30km/h, 50km/h, 70km/h) an light conditions (day, sunset, foggy).

In [14]:
%pip install supervision
import supervision as sv
import numpy as np
from ultralytics import YOLO
import cv2

VIDEO_PATH = "/kaggle/input/citroen/citroen.mp4"
TARGET_PATH = "/kaggle/working/inference/selfdrive-tracking-citroen-2.mp4"
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")


class VideoInference(object):
    def __init__(self, video_path: str, model):
        self._video_path = video_path
        self._model = model
        self._byte_tracker = sv.ByteTrack()
        
    def adjust_brightness(self, image, brightness_factor):
        # Convert image to HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # Scale the V (brightness) channel
        hsv[:,:,2] = np.clip(hsv[:,:,2] * brightness_factor, 0, 255)
        # Convert the image back to BGR color space
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
    def run(self, target_path: str):
        annotator = sv.BoxAnnotator()
        video_info = sv.VideoInfo.from_video_path(self._video_path)
        
        def callback(frame: np.ndarray, index: int) -> np.ndarray:
            # Detect objects in the frame
            results = model(frame, conf=0.4)[0]
            detections = sv.Detections.from_ultralytics(results)
            detections = self._byte_tracker.update_with_detections(detections)

            # Adjust brightness of the entire frame (background)
            frame = self.adjust_brightness(frame, brightness_factor=0.5)  # Decrease brightness

            # Create a mask for detected objects
            mask = np.zeros_like(frame[:, :, 0], dtype=np.uint8)
            for detection in detections:
                x1, y1, x2, y2 = detection[0]
                # print(x1, y1, x2, y2)
                cv2.rectangle(mask, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 255), -1)

            # Adjust brightness of the detected objects (contours)
            contour_brightness_factor = 1.5  # Increase brightness
            frame_contours = self.adjust_brightness(frame, brightness_factor=contour_brightness_factor)

            # Combine the frames with adjusted brightness for background and contours
            frame_with_contours = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(mask))
            frame_with_contours += cv2.bitwise_and(frame_contours, frame_contours, mask=mask)

            labels = [
                f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
               for _, _, confidence, class_id, tracker_id, _
              in detections
            ]
            
            return annotator.annotate(scene=frame_with_contours.copy(), detections=detections, labels=labels)

        sv.process_video(source_path=VIDEO_PATH, target_path=target_path, callback=callback)

video_inference = VideoInference(VIDEO_PATH, model)
video_inference.run(TARGET_PATH)

Note: you may need to restart the kernel to use updated packages.

0: 384x640 5 cars, 1 trafficLight-Red, 81.9ms
Speed: 3.6ms preprocess, 81.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 trafficLight-Red, 78.7ms
Speed: 3.0ms preprocess, 78.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 trafficLight-Red, 73.7ms
Speed: 2.3ms preprocess, 73.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 trafficLight-Red, 71.5ms
Speed: 2.5ms preprocess, 71.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 trafficLight-Red, 75.8ms
Speed: 2.4ms preprocess, 75.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 trafficLight-Red, 77.3ms
Speed: 2.6ms preprocess, 77.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 trafficLight-Red, 77.9ms
Speed: 2.4ms preprocess

### 2. License plate detection model

Now we have **ultralytics** installed, so we only need to create a new model. This model will be trained using a dataset of around 20k annotated images with license plates and will be applied on ROI extracted using first model.

This model has already been trained so now we only load best obtained weights.

In [ ]:
from ultralytics import YOLO
import torch

print(torch.cuda.is_available())

model = YOLO("/kaggle/working/runs/detect/train9/weights/best.pt")
model.cuda()